In [6]:
# !pip install kss
# !pip install py-hanspell
!pip install emoji==1.2.0

In [1]:
import time, re
import pandas as pd
import emoji
from kss import split_sentences 
import warnings
warnings.filterwarnings(action='ignore') 

In [63]:
# load reviews 
with open('C:/Users/olive_young/reviews.txt') as f:
    reviews = [line.rstrip() for line in f]

print(f"reviews size : {len(reviews)}")

reviews size : 13112


In [64]:
reviews

['프리메라 유스 래디언스 비타레놀 세럼은 2달정도 사용한것 같아요',
 '피토바이오틱스와 비타민 C60%, 레티놀이 함유되어 있는 세럼이라서 좋아하는 세럼입니다',
 '15그램에 가격은 비싸지만 쿠폰 사용해서 구매하고 레티놀이 함유되어 있어서 꼭 아침에 바를 시 선크림을 발라야 해서 항상 챙겨 바를 자신이 없어서 밤에만 바르고 있어요',
 '전 레티놀은 1년 정도 발라서 적응기간은 끝나서 자극은 하나도 없게 느껴져서 처음 레티놀제품을 사용하시는 분은 어떨지 모르겠네요 레티놀은 일주일 정도 적응기간을 거친 후 사용하는건데 이 세럼은 다른 기능성도 혼합된거라서 괜찮을 것 같기도 해요',
 '처음 사용시 펌프를 눌러서 혼합한 후에 5분정도 둔 다음에 사용해야 됩니다',
 '펌프를 누르면 비타민 C 캡슐이 내려와 레티놀 세럼과 혼합하여 사용하는 제품이거든요',
 '전 세안 후 토너 사용후 히알루론산 앰플 바르고 이 비타레놀 세럼 사용하고 탄력 세럼 아이크림 콜라겐크림 순으로 바르고 있어요 레티놀과 비타민 C 는 40대인 저에겐 없어서는 안되는 제품인데 혼합된 제품이 나와서 너무나 좋습니다',
 '🖤재구매 사용 후기에요!',
 '🖤첫사용시 딸깍하는 소리날 때까지 꾹 누른다음에 엄청 흔들어주고 사용하는거 주의하세요!!!!!!!',
 '🖤수부지, 홍조, 여드름, 민감성 피부에도 자극이 하나도 없었어요.',
 '보통 레티놀, 비타민 조합들은 따가워서 사용하기 어려웠는데',
 '프리메라 비타티놀은 트러블이 하나도 없습니다!!!',
 '🖤예민한 피부들에게 비타민 들어간 제품 추천하라면 무조건 비타티놀..! 왜냐...! 제가 이제까지 비타민 들어간 화장품 죄다 따갑고 트러블나서 못써봤는데 이건 트러블 하나도 없고 오히려 안색이 밝아져요!',
 '🖤한통 다 써보고, 재구매를 하게 된건 단지 트러블이 안나서가 아니라',
 '한통 다 쓰고 한달동안 안색이 점점 좋아지는게 보이더라고요!',
 '안쓰니까 다시 피부에 탄력이 없어지는 기분이라 재구매함 ㅠ',
 '🖤솔직히 적은 용량에 가격

In [41]:
# load reviews 
test_reviews_raw = pd.read_csv('E:/olive_young/Review_NLP/dataset/test_reviews_raw.csv')
test_reviews_raw

,Unnamed: 0,review,list_review
0,0,💜구매동기💜,0
1,1,웰라쥬는 원래 수분라인 제품으로 유명하다고 알고 있어서 진정제품은 잘 알지 못했는데...,0
2,2,💜사용후기💜,0
3,3,일단 사용후기는 정말 만족도 최상입니다!!!,0
4,4,제가 원래 그날 기간에는 얼굴이 완전 뒤집어지고 피부 컨디션도 안좋아지는데 이 크림...,0
...,...,...,...
5942,7174,처음에 썼을 때는 이마에 있는 여드름이 많이 가라앉았었는데 일시적일 뿐 다시 올라오...,29
5943,7175,볼쪽에도 쓰기전에는 많이 없었는데 이거 바르고 나니깐,29
5944,7176,너무 올라와서 원래 쓰던 거만 쓰니까 바로 가라앉더라고요ㅋㅋ,29
5945,7177,제 피부한테 안맞는 거 겠지만 후기올려봅니다,29


In [43]:
split_reviews = []
for i in range(30):
    txt = test_reviews_raw.loc[test_reviews_raw.list_review == i].review
    tup = (list(txt), i)
    split_reviews.append(tup)
len(split_reviews)

30

In [ ]:
def write_file(list_data):
    with open('test_reviews_raw.txt', 'wt') as f:
        f.write('\n'.join(list_data))
    return

In [82]:
print(f"reviews size : {len(test_reviews_raw)}")

reviews size : 5947


### 리뷰 전처리
* 이모티콘 제거
* 문장 분리 
* 문장 내 불필요한 특수 문자 제거 
* 특정 문자 다른 문자로 치환 
* 문장 맨 앞 노이즈 문자 제거 (접속사 등)
* 문장 맨 앞 숫자 제거 
* 문장 맨 앞 빈 문자열 제거
* 문장 중간 두 칸 이상 띄워쓰기 제거
* '주제: ' 패턴 지우기

In [44]:
def remove_multispaces(input_str) :
    len_prev = len(input_str)
    
    while True :
        input_str = re.sub('[ ]+', ' ', input_str)
        len_cur = len(input_str)
        
        if len_prev >= len_cur :
            break
        else :
            len_prev = len_cur
    
    return input_str.strip()

def remove_emoji(input_str):
    input_str = emoji.get_emoji_regexp().sub(u'', input_str)
    return remove_multispaces(input_str)

def remove_chars(input_str, pattern): 
    input_str = re.sub(pattern, '', input_str)
    return remove_multispaces(input_str)

def replace_chars(input_str, pattern, replace_str):
    input_str = re.sub(pattern, replace_str, input_str)
    return remove_multispaces(input_str)

def remove_bracket_edge(input_str) :
    if len(input_str) >= 2 :
        if input_str[0] =='(' and input_str[-1] ==')':
            input_str = input_str[1:-1]
    
    return input_str.strip()

In [3]:
patterns = [':-\)', '\([^)]*\)', '\[[^)]*\]', '\<[^)]*\>','[^:]*:',
           '.\)', '[0-9]+[\. ]', ':D', ':\(', '[,.]*',
            '[!_ㅠㅜㅎㅋ^\[\]<>\'\"“‘;★☆♤♡♥※♧♣♤♠■□◇◆◈▣△▲▽▼◁◀▷▶○●⊙◐◑◎]*']

def remove_patterns(sent) :
    for pattern in patterns :
        sent = remove_chars(sent, pattern)
    
    return sent        

In [1]:
start_words = ["BUT", "그런데", "그런대", "그리고", "근데", "근대", "그래도", "그런만큼", "그래서", "구래서", "그렇다고",
               "그리구", "글고", "그치만", "그러다가", "그러고", "그렇지만", "그랬더니", "그러데", "아니면", "그런지", "일단"
               "그만큼", "그러나", "때문에", "앞에 언급했던것 것처럼", "그래두", "즉,", "되려", "덧붙여", "더불어", "이것도",
               "그러니", "그러다", "그것빼곤", "그러고", "그래가지고", "그렇다보니", "그렇다 보니", "그럼에도", "그러므로", 
               "따라서", "그런 점에서", "그러면", "그러니까", "그래서인지", "그만큼", "다만", "대신", "결론은", "대신에", "인지 ",
               "암튼간에", "아무튼", "암튼", "여튼", "무튼", "하지만", "오히려", "물론", "게다가", "또 ", "특히", "이게", "이번에",
               "특히나", "이래서", "단 ", "단," "우선", "이 외에도", "뭐근데", "그러다가", "덕분에", "그 덕에", "결국은",  
               "결국", "결론적으로", "결과적으로", "참고로", "심지어", "왜냐하면",  "이외에도", "반대로","더구나", "반면에", 
               "즉 ", "마지막으로", "혹은", "아무쪼록", "일단", "일단은"," 거기에" "먼저", "어쨌든", "쨌든", "어쨋든", "쨋든", 
               "또는", "앞서 말했듯이", "더더군다나", "거기다", "거기다가", "그것 외에는", "그밖에는", "그 밖에는", "참고로", 
               "왜냐", "그에 반해", "그 외엔", "그외엔", "그외에", "그 외에", "지금처럼", "대신에", "더불어", "오히려",  "물론", 
               "다만", "오우", "아 ", "오 ", "와 ", "헐 ", "어 ", "음 ", "으음", "옹 ", "->", "→", "☆", "\+", "@", "#", 
               "•", "\)", "\ㅣ","✓", "0", "○", "\.", "," "=" , "=>", "&", "/", "➡", "➤", "⇨", "☞", " ☛", ":", "-", "~", 
               "#", "\*", "•", "…"]

def remove_start_words(sent) :
    for start_word in start_words :
        len_prev = len(sent)
        
        while True :
            sent = re.sub(pattern=f'^{start_word}', repl='', string=sent).strip()
            len_cur = len(sent)
            
            if len_prev >= len_cur :
                break
            else :
                len_prev = len_cur
    
    return sent.strip()

In [48]:
def preprocessing(review) :
    result = []
    
    # 하나의 문장으로 연결되어 있는 리뷰 전체를 문장 분할
    sents = split_sentences(review)
    
    # 문장 분할 후, 문장 단위로 텍스트 전처리 수행
    for idx, sent in enumerate(sents) :
        sent = remove_emoji(sent) # 이모지 제거
        sent = remove_bracket_edge(sent) # 문장 전체가 괄호라면, 괄호 기호만 삭제
        sent = remove_patterns(sent) # 정규 표현식 패턴을 이용한 일괄 제거
        sent = remove_start_words(sent) # 시작 불용어 제거
        sent = replace_chars(sent, '&', ', ')
        
        if len(sent) > 4 :
            result.append(sent)
    
    return result

In [49]:
def reviews_prepro(reviews_list):
    sents_all = []
    for review in reviews_list :
        review = review.strip()
        if len(review) == 0 :
            continue

        sents = preprocessing(review)
        sents_all.extend(sents)

    return list(set(sents_all))

In [54]:
clean_reviews_total = []

for i in range(30):
    temp = reviews_prepro(reviews_prepro(split_reviews[i][0]))
    clean_reviews_total.append((temp, split_reviews[i][0]))

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363



In [55]:
len(clean_reviews_total)

30

In [69]:
pd.DataFrame(clean_reviews_total).to_csv('./test_test_reviews.csv')

In [71]:
with open('test_test_reviews.pickle', 'wb') as f:
    pickle.dump(clean_reviews_total, f)

In [90]:
reviews_df = pd.DataFrame(clean_reviews)
reviews_df.columns = ['reviews']
reviews_df ['key_sent'] = 0
reviews_df ['sentiment'] = 0
reviews_df

,reviews,key_sent,sentiment
0,웰라쥬는 원래 수분라인 제품으로 유명하다고 알고 있어서 진정제품은 잘 알지 못했는데...,0,0
1,사용후기는 정말 만족도 최상입니다,0,0
2,제가 원래 그날 기간에는 얼굴이 완전 뒤집어지고 피부 컨디션도 안좋아지는데 이 크림...,0,0
3,아예 안나지는 않는데 23개 뾰루지 올라오는가보면 효과 짱인것같아요,0,0
4,씻고 나오면 얼굴이 항상 붉었는데 붉은기도 가라앉아서 얼굴이 다 환해보여요,0,0
...,...,...,...
9133,처음에 썼을 때는 이마에 있는 여드름이 많이 가라앉았었는데 일시적일 뿐 다시 올라오...,0,0
9134,볼쪽에도 쓰기전에는 많이 없었는데 이거 바르고 나니깐,0,0
9135,너무 올라와서 원래 쓰던 거만 쓰니까 바로 가라앉더라고요,0,0
9136,제 피부한테 안맞는 거 겠지만 후기올려봅니다,0,0


In [91]:
reviews_df.to_csv('E:/olive_young/Review_NLP/test_reviews_prepro.csv')